In [ ]:
# plan
# 1. so far the listing data we have are all for used cars
# 2. can make use of the used_car ID to search for review link for a specific car model

In [1]:
# import packages
import bs4
from bs4 import BeautifulSoup
import requests as req
import pickle
import re
import pandas as pd
import numpy as np

In [ ]:
# scrape review text
def scrape_review_data(url):
    
    # check running number
    print(url.split("=")[1])
    
    # Step1: form beautifulSoup from url
    html_text = req.get(url)
    soup = BeautifulSoup(html_text.content, "lxml")

    # Step2: get entire review page content
    review_text = soup.body.find('div', {'id':'content'}).text
    
    # Step3: clean the raw html text and hold it inside a temp bag as list
    # Note: \xa0 is a line break from html text
    review_raw = review_text.strip().replace('\s', '').split('\xa0')
    review_tempBag = []
    for text in review_raw:
        if re.match("^[a-zA-Z]", text):
            review_tempBag.append(text.replace('\n', ' '))
    
    # Step4: return cleaned review text
    review_cleaned = ' '.join(review_tempBag)

    # Step5: extract relevant information
    # 1. Model
    model = review_cleaned.partition('User Reviews')[2].partition('Price')[0].strip()
    # 2. Review
    review = review_cleaned
#     # 2. What We Like
#     review_like = review_cleaned.partition('What We Like')[2].partition('What We Dislike')[0].strip()
#     # 3. What We Dislike
#     review_dislike = review_cleaned.partition('What We Dislike')[2].partition('The')[0].strip()
    
    return model, review

In [2]:
# form a list of url
url_list = [r'https://www.sgcarmart.com/news/review.php?AID=%d' %(AID+1) for AID in range(2068)]
print(url_list[0])
len(url_list)

https://www.sgcarmart.com/news/review.php?AID=1


2068

In [ ]:
# check sample return
# NOTE: map can only release content once. Need to rerun after convert to list
result = map(scrape_review_data, [url_list[0]])
result = list(result)

In [ ]:
models = []
reviews = []
failed_attempt = 0
failed_ID = []
for url in url_list:
    try:
        result = scrape_review_data(url)
        models.append(result[0])
        reviews.append(result[1])
    except:
        failed_attempt += 1
        failed_ID.append(url.split("=")[1])
        pass

In [ ]:
# convert lists to data frame
df_review = pd.DataFrame(zip(models, reviews), columns=['model', 'review_text'])
df_review.head()

In [ ]:
df_review.head()

In [ ]:
# process review text to extract like and dislike comments
df_review['review_like'] = df_review['review_text'].apply(lambda x : str(x).partition('What We Like')[2].partition('What We Dislike')[0].strip())
df_review['review_dislike'] = df_review['review_text'].apply(lambda x : str(x).partition('What We Dislike')[2].partition('The')[0].strip())
df_review.loc[df_review.review_like == "", 'review_like'] = None
df_review.loc[df_review.review_dislike == "", 'review_dislike'] = None

df_review.head()

In [ ]:
df_review.info()

In [ ]:
# save the dataframe as csv
df_review.to_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/SGCarMart_reviewdata.csv', index=True)

In [ ]:
######################################
#####      test environment      #####
##### to scrape rating hyperlink #####
######################################

def car_reviews_href(href):
    return href and re.compile(r"(^(newcars_reviews.php).*)").search(href)

url = r'https://www.sgcarmart.com/news/review.php?AID=2068'

html_text = req.get(url)
soup = BeautifulSoup(html_text.content, 'lxml')

rating_href = soup.body.find('div', {'id':'contentblank'}).find('div', {'id':'rightside_content'}).find('div', {'id':'car_information'}).find('div', {'class':'greybox_body newboxborder'}).find('div', {'class':'endside'}).select('div:nth-of-type(2)')[0].select('div:nth-of-type(1)')[0].find('table').find_all('a', string=True)
rating_href = 'sgcarmart.com' + str(rating_href[0]).partition('href=')[2].partition('>')[0].replace('"', "")
print(rating_href.find('CarCode') > 0)

In [11]:
# define a function to get review rating url
def get_reviewrating_href(url):
    
    try:
        # construct BeautifulSoup
        html_text = req.get(url)
        soup = BeautifulSoup(html_text.content, 'lxml')

        # locate a-tag and extract as string
        rating_href = soup.body.find('div', {'id':'contentblank'}).find('div', {'id':'rightside_content'}).find('div', {'id':'car_information'}).find('div', {'class':'greybox_body newboxborder'}).find('div', {'class':'endside'}).select('div:nth-of-type(2)')[0].select('div:nth-of-type(1)')[0].find('table').find_all('a', string=True)
        # verify if the a-tag is a valid link for review rating
        if str(rating_href).find('CarCode') > 0:
            rating_href = 'sgcarmart.com' + str(rating_href[0]).partition('href=')[2].partition('>')[0].replace('"', "")

        return rating_href
    except:
        print(url)
        return None
        pass

In [14]:
# map the function to url_list
reviewRating_links = list(map(get_reviewrating_href, url_list))

https://www.sgcarmart.com/news/review.php?AID=14
https://www.sgcarmart.com/news/review.php?AID=15
https://www.sgcarmart.com/news/review.php?AID=17
https://www.sgcarmart.com/news/review.php?AID=22
https://www.sgcarmart.com/news/review.php?AID=37
https://www.sgcarmart.com/news/review.php?AID=56
https://www.sgcarmart.com/news/review.php?AID=57
https://www.sgcarmart.com/news/review.php?AID=58
https://www.sgcarmart.com/news/review.php?AID=60
https://www.sgcarmart.com/news/review.php?AID=61
https://www.sgcarmart.com/news/review.php?AID=65
https://www.sgcarmart.com/news/review.php?AID=67
https://www.sgcarmart.com/news/review.php?AID=68
https://www.sgcarmart.com/news/review.php?AID=69
https://www.sgcarmart.com/news/review.php?AID=76
https://www.sgcarmart.com/news/review.php?AID=82
https://www.sgcarmart.com/news/review.php?AID=83
https://www.sgcarmart.com/news/review.php?AID=85
https://www.sgcarmart.com/news/review.php?AID=90
https://www.sgcarmart.com/news/review.php?AID=93
https://www.sgcarmar

https://www.sgcarmart.com/news/review.php?AID=1216
https://www.sgcarmart.com/news/review.php?AID=1217
https://www.sgcarmart.com/news/review.php?AID=1218
https://www.sgcarmart.com/news/review.php?AID=1219
https://www.sgcarmart.com/news/review.php?AID=1222
https://www.sgcarmart.com/news/review.php?AID=1223
https://www.sgcarmart.com/news/review.php?AID=1226
https://www.sgcarmart.com/news/review.php?AID=1231
https://www.sgcarmart.com/news/review.php?AID=1237
https://www.sgcarmart.com/news/review.php?AID=1241
https://www.sgcarmart.com/news/review.php?AID=1243
https://www.sgcarmart.com/news/review.php?AID=1246
https://www.sgcarmart.com/news/review.php?AID=1252
https://www.sgcarmart.com/news/review.php?AID=1253
https://www.sgcarmart.com/news/review.php?AID=1256
https://www.sgcarmart.com/news/review.php?AID=1264
https://www.sgcarmart.com/news/review.php?AID=1266
https://www.sgcarmart.com/news/review.php?AID=1271
https://www.sgcarmart.com/news/review.php?AID=1272
https://www.sgcarmart.com/news/

In [142]:
len(reviewRating_links)

2068

In [91]:
cleaned_links = [None if link == [] else link for link in reviewRating_links]

In [92]:
# save the href link to csv file
valid_links = pd.DataFrame(cleaned_links, columns=['rating_href'])
valid_links.rating_href = valid_links.rating_href.apply(lambda x: 'https://www.'+str(x) if x != None else None)
valid_links.to_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/SGCarMart_reviewRatingLinks.csv', index=True)

In [84]:
######################################
#####      test environment      #####
##### to scrape rating parameter #####
######################################
url = valid_links.rating_href[0]

html_text = req.get(url)
soup = BeautifulSoup(html_text.content, 'lxml')

# extract all ratings respectively
overall = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(2)')[0].find('table').find('tr').find('div').select('div:nth-of-type(1)')[0].text
print(overall.strip())
comfort = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(1)')[0].select('td:nth-of-type(3)')[0].text
print(comfort)
reliability = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(1)')[0].select('td:nth-of-type(6)')[0].text
print(reliability)
handling = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(3)')[0].select('td:nth-of-type(3)')[0].text
print(handling)
features = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(3)')[0].select('td:nth-of-type(6)')[0].text
print(features)
exterior_styling = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(5)')[0].select('td:nth-of-type(3)')[0].text
print(exterior_styling)
interior_design = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(5)')[0].select('td:nth-of-type(6)')[0].text
print(interior_design)
value_for_money = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(7)')[0].select('td:nth-of-type(3)')[0].text
print(value_for_money)
engine_performance = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(7)')[0].select('td:nth-of-type(6)')[0].text
print(engine_performance)


3.0
3.5
2.5
3.0
2.5
3.0
3.0
3.5
2.5


In [130]:
# extract ratings
# logics: if sample link from result is not None, extract the rating
overall_list = []
comfort_list = []
reliability_list = []
handling_list = []
features_list = []
exterior_styling_list = []
interior_design_list = []
value_for_money_list = []
engine_performance_list = []

In [131]:
def extract_review_rating(url):
    
    print(url)
    
    # set default rating as None
    overall, comfort, reliability, handling, features, exterior_styling, interior_design, value_for_money, engine_performance = None, None, None, None, None, None, None, None, None
    
    if url != None:
        html_text = req.get(url)
        soup = BeautifulSoup(html_text.content, 'lxml')
        try:
            overall = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(2)')[0].find('table').find('tr').find('div').select('div:nth-of-type(1)')[0].text.strip()
            comfort = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(1)')[0].select('td:nth-of-type(3)')[0].text.strip()
            reliability = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(1)')[0].select('td:nth-of-type(6)')[0].text.strip()
            handling = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(3)')[0].select('td:nth-of-type(3)')[0].text.strip()
            features = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(3)')[0].select('td:nth-of-type(6)')[0].text.strip()
            exterior_styling = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(5)')[0].select('td:nth-of-type(3)')[0].text.strip()
            interior_design = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(5)')[0].select('td:nth-of-type(6)')[0].text.strip()
            value_for_money = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(7)')[0].select('td:nth-of-type(3)')[0].text.strip()
            engine_performance = soup.body.find('div', {'id':'content'}).find('div', {'id':'contentblank'}).find('div', {'id':'maincontent'}).find('div', {'class':'mainright'}).find('div', {'class':'mainrightblank'}).select('div:nth-of-type(7)')[0].find('div',{'class':'newboxborder'}).next_sibling.next_sibling.find('table').find('tr').select('td:nth-of-type(4)')[0].find('table').select('tr:nth-of-type(7)')[0].select('td:nth-of-type(6)')[0].text.strip()
            
            # add the value to list
            overall_list.append(float(overall))
            comfort_list.append(float(comfort))
            reliability_list.append(float(reliability))
            handling_list.append(float(handling))
            features_list.append(float(features))
            exterior_styling_list.append(float(exterior_styling))
            interior_design_list.append(float(interior_design))
            value_for_money_list.append(float(value_for_money))
            engine_performance_list.append(float(engine_performance))
            
        except:
            print("EXCEPTION")
            overall, comfort, reliability, handling, features, exterior_styling, interior_design, value_for_money, engine_performance = None, None, None, None, None, None, None, None, None
            overall_list.append(overall)
            comfort_list.append(comfort)
            reliability_list.append(reliability)
            handling_list.append(handling)
            features_list.append(features)
            exterior_styling_list.append(exterior_styling)
            interior_design_list.append(interior_design)
            value_for_money_list.append(value_for_money)
            engine_performance_list.append(engine_performance)
            pass
        
    else:
        overall_list.append(overall)
        comfort_list.append(comfort)
        reliability_list.append(reliability)
        handling_list.append(handling)
        features_list.append(features)
        exterior_styling_list.append(exterior_styling)
        interior_design_list.append(interior_design)
        value_for_money_list.append(value_for_money)
        engine_performance_list.append(engine_performance)
    
    

In [132]:
result = map(extract_review_rating, list(valid_links.rating_href))
list(result)

https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10504
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10038
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10469
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10485
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10488
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10150
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10323
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10511
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10442
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10515
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10475
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10502
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10445
None
None
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?Car

https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10815
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10575
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10808
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10806
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10356
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10807
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10015
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10286
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10183
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10268
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10803
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10826
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10686
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10321
None
None
https://w

EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10996
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10948
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11015
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11000
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10870
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11013
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10764
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10801
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10515
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10797
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11001
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11010
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10358
None
https://www.sgcarmart.com/new_cars/newcars_reviews.ph

EXCEPTION
None
None
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11217
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11193
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11111
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11060
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10713
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11263
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11264
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10870
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11144
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11260
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11275
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10616
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11262
https://www.sgcarmart.com/new_cars/newcars_

None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10784
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11446
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11461
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11451
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11491
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11495
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=10784
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11375
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11069
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11487
EXCEPTION
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11494
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11508
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11487
EXCEPTION
None
None
None
https://www.sgcarmart.c

...

https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12508
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11857
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12363
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12427
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12406
EXCEPTION
None
None
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12265
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12331
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12423
EXCEPTION
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11789
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12396
None
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11650
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12482
None
None
https://www.sgcarmart.com/new_ca

EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12233
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12654
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12545
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12725
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12728
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12561
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12267
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12651
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12117
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=11603
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12626
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12545
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12620
EXCEPTION
https://

EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12889
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12893
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12812
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12815
EXCEPTION
None
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12653
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12888
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12827
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12476
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12916
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12903
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12893
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12643
EXCEPTION
https://www.

EXCEPTION
None
None
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=21518
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12922
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12897
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12128
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12895
EXCEPTION
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12979
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12922
EXCEPTION
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12665
EXCEPTION
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12648
EXCEPTION
None
None
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=21510
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12247
EXCEPTION
https://www.sgcarmart.com/new_cars/newcars_reviews.php?C

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [141]:
sample = list(result)
list(valid_links.rating_href).index('https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12962')
list(valid_links.rating_href)[1842]

'https://www.sgcarmart.com/new_cars/newcars_reviews.php?CarCode=12947'

In [133]:
# read in the saved reviews and ratings datasets
ratings = pd.DataFrame(zip(overall_list, comfort_list, reliability_list, handling_list, features_list, exterior_styling_list, interior_design_list, value_for_money_list, engine_performance_list), 
                       columns = ['overall', 'comfort', 'reliability', 'handling', 'features', 'exterior_styling', 'interior_design', 'value_for_money', 'engine_performance'])
ratings.to_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/SGCarMart_reviewratings.csv')

In [134]:
# combine the review text and rating into one dataframe
reviews = pd.read_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/SGCarMart_reviewdata.csv', usecols=lambda x: x != 'Unnamed: 0')
ratings = pd.read_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/SGCarMart_reviewratings.csv', usecols=lambda x: x != 'Unnamed: 0')

In [135]:
reviews.head()

,model,review_text,review_like,review_dislike
0,Geely MK 1.5 (M),All Articles Car News Car Reviews Car Advice F...,NaN,NaN
1,Daihatsu Sirion 1.5 Sports (A),All Articles Car News Car Reviews Car Advice F...,NaN,NaN
2,Fiat Bravo 1.4 T-Jet Dynamic (M),All Articles Car News Car Reviews Car Advice F...,NaN,NaN
3,Mazda 2 Hatchback 1.5 V (A),All Articles Car News Car Reviews Car Advice F...,NaN,NaN
4,Suzuki SX4 Sedan 1.6 (A),All Articles Car News Car Reviews Car Advice F...,NaN,NaN


In [136]:
ratings.head()

,overall,comfort,reliability,handling,features,exterior_styling,interior_design,value_for_money,engine_performance
0,3.0,3.5,2.5,3.0,2.5,3.0,3.0,3.5,2.5
1,3.5,3.5,3.5,3.0,3.0,3.5,3.0,4.0,3.5
2,4.0,4.0,3.5,4.0,3.5,4.0,4.0,3.5,4.0
3,4.0,4.0,4.0,4.0,4.0,4.0,3.5,3.5,3.5
4,4.0,4.5,4.0,4.0,3.5,3.5,4.0,4.0,3.5


In [137]:
# concatenate the two datasets
df = pd.concat([reviews, ratings], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   model               981 non-null    object 
 1   review_text         2068 non-null   object 
 2   review_like         711 non-null    object 
 3   review_dislike      673 non-null    object 
 4   overall             894 non-null    float64
 5   comfort             894 non-null    float64
 6   reliability         894 non-null    float64
 7   handling            894 non-null    float64
 8   features            894 non-null    float64
 9   exterior_styling    894 non-null    float64
 10  interior_design     894 non-null    float64
 11  value_for_money     894 non-null    float64
 12  engine_performance  894 non-null    float64
dtypes: float64(9), object(4)
memory usage: 210.2+ KB


In [138]:
# export the combined dataset
df.to_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/SGCarMart_Review_Rating.csv')